<a href="https://colab.research.google.com/github/EdwardMatsuzaka/Evidencia1/blob/main/EV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importar librerias a usadas en el código
import sys
import datetime
from datetime import date, datetime
import re
import openpyxl
import csv
import sqlite3
from sqlite3 import Error

#Def creados para el programa

#Crear Reportes Tabulares
def crear_reporte_tabular(registros):
    # Imprimir encabezados de las columnas
    print("=" * 170)
    print(f"| Clave |\t Título \t|\t Autor \t\t |\t Género \t|\t Año de Publicación \t|\t ISBN\t\t|\t Fecha Adquisición \t|")
    print("=" * 170)
    
    # Imprimir registros en formato de tabla
    for clave, titulo, autor, genero, año_publicacion, isbn, fecha_adquisicion in registros:
        print(f"| {clave:^5} | {titulo:^21} | {autor:^22} | {genero:^20} | {año_publicacion:^29} | {isbn:^21} | {fecha_adquisicion:^29} |")
    
    print("=" * 170)


#Crear tabla para ejemplares
def ejemplares():
  try:
    with sqlite3.connect("EV3_EJEMPLARES.db") as conn:
      cursor = conn.cursor()
      cursor.execute("CREATE TABLE IF NOT EXISTS TABLA_EJEMPLARES \
      (clave INTEGER PRIMARY KEY, titulo TEXT NOT NULL, \
      AUTORES INTEGER NOT NULL, GENERO INTEGER NOT NULL, \
      año_publicacion TEXT NOT NULL, isbn TEXT NOT NULL, fecha_adquisicion TEXT NOT NULL, \
      FOREIGN KEY(AUTORES) REFERENCES AUTORES(clave_autor), \
      FOREIGN KEY(GENERO) REFERENCES GENERO(clave_genero));")

      cursor.execute("CREATE TABLE IF NOT EXISTS AUTORES \
      (clave_autor INTEGER PRIMARY KEY, apellido TEXT NOT NULL, nombre TEXT NOT NULL);")

      cursor.execute("CREATE TABLE IF NOT EXISTS GENERO \
      (clave_genero INTEGER PRIMARY KEY, genero TEXT NOT NULL);")

      print("---Tablas creadas exitosamente---")
  except Error as e:
      print(e)
  except Exception:
      print(f"**Se produjo el siguiente error: {sys.exc_info()[0]}**")

#Exportar en csv el Catálogo de Ejemplares
def ejemplares_csv():
  try:
    with open('catalogo_ejemplares.csv', 'w', newline='') as csv_catalogo_ejemplares:
      writer = csv.writer(csv_catalogo_ejemplares)
      writer.writerow(['Clave', 'Titulo', 'Autor', 'Genero', 'ISBN', 'Año de Publicación', 'Fecha de Adquisicion'])
      for registro in registros:
        writer.writerow(registro)
      print("--- Se ha guardado el Catálogo Completo en el archivo 'catalogo_ejemplares.csv' ---")
  except sqlite3.Error as e:
    print(e)
  except Exception:
    print(f"**Se produjo el siguiente error: {sys.exc_info()[0]}**")
  finally:
    if (conn):
      conn.close()
      print("--- Se ha cerrado la conexión ---")

#Crear excel del Catálogo completo de los ejemplares
def ejemplares_excel():
  libro = openpyxl.Workbook()
  hoja = libro.active
  hoja.title = "Todos los ejemplares"

  hoja["A1"].value = "CLAVE"
  hoja["B1"].value = "TÍTULO"
  hoja["C1"].value = "AUTOR"
  hoja["D1"].value = "GÉNERO"
  hoja["E1"].value = "ISBN"
  hoja["F1"].value = "AÑO PUBLICACION"
  hoja["G1"].value = "FECHA"

  from openpyxl.styles import Font
  celda_negritas = hoja["A1":"G1"]
  negritas = Font(bold = True)
  for fila in celda_negritas:
    for celda in fila:
      celda.font = negritas


  # Agregar los valores a las celdas
  filas = 2
  for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros:
      hoja.cell(row=filas, column=1).value = clave
      hoja.cell(row=filas, column=2).value = titulo
      hoja.cell(row=filas, column=3).value = autor
      hoja.cell(row=filas, column=4).value = genero
      hoja.cell(row=filas, column=5).value = isbn
      hoja.cell(row=filas, column=6).value = año_publicacion
      hoja.cell(row=filas, column=7).value = fecha_adquisicion
      filas += 1

  # Guardar el archivo de Excel
  libro.save("catalogo_completo_ejemplares.xlsx")


#Crear excel del Reporte por autor
def excel_autor():
  ejemplares_por_autor=[]
  for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros_autor:
    if autor == consulta_reporte:
      ejemplares_por_autor.append((clave, titulo, autor, genero, isbn, año_publicacion,  fecha_adquisicion))

      crear_reporte_tabular(ejemplares_por_autor)
      
      if not ejemplares_por_autor:
        print(f"No se encontraron ejemplares del autor {consulta_reporte}")
  libro = openpyxl.Workbook()
  hoja = libro["Sheet"]
  hoja.title = f"Ejemplares de {consulta_reporte}"

  hoja["A1"].value = "CLAVE"
  hoja["B1"].value = "TÍTULO"
  hoja["C1"].value = "AUTOR"
  hoja["D1"].value = "GÉNERO"
  hoja["E1"].value = "ISBN"
  hoja["F1"].value = "AÑO PUBLICACION"
  hoja["G1"].value = "FECHA"

  for row_num, ejemplar in enumerate(ejemplares_por_autor, 2):
    for col_num, valor in enumerate(ejemplar, 1):
        hoja.cell(row=row_num, column=col_num, value=valor)

  nombre_archivo = f"ejemplares_por_autor_{consulta_reporte}.xlsx"
  libro.save(nombre_archivo)
  print(f"Archivo guardado como {nombre_archivo}")

#Crear excel del Reporte por Género
def excel_genero():
  ejemplares_por_genero=[]
  for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros_generos:
    if genero == consulta_reporte:
      ejemplares_por_genero.append((clave, titulo, autor, genero, isbn, año_publicacion,  fecha_adquisicion))
      print(f"=Clave: {clave}\n=Titulo: {titulo}\n=Autor: {autor}\n=Genero: {genero}\n=Año de Publicación: {año_publicacion}\n=ISBN: {isbn}\n=Fecha: {fecha_adquisicion}")
      print(f"-"*30)  
      if not ejemplares_por_genero:
        print(f"No se encontraron ejemplares del género {consulta_reporte}")
  libro = openpyxl.Workbook()
  hoja = libro["Sheet"]
  hoja.title = f"Ejemplares de {consulta_reporte}"

  hoja["A1"].value = "CLAVE"
  hoja["B1"].value = "TÍTULO"
  hoja["C1"].value = "AUTOR"
  hoja["D1"].value = "GÉNERO"
  hoja["E1"].value = "ISBN"
  hoja["F1"].value = "AÑO PUBLICACION"
  hoja["G1"].value = "FECHA"

  for row_num, ejemplar in enumerate(ejemplares_por_genero, 2):
    for col_num, valor in enumerate(ejemplar, 1):
        hoja.cell(row=row_num, column=col_num, value=valor)

  nombre_archivo = f"ejemplares_por_genero_{consulta_reporte}.xlsx"
  libro.save(nombre_archivo)
  print(f"Archivo guardado como {nombre_archivo}")

#Crear Excel del Reporte por Año Publicación
def excel_año_publicacion():
  ejemplares_por_año_publicacion=[]
  for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros_año_publicacion:
    if año_publicacion == consulta_reporte:
      ejemplares_por_año_publicacion.append((clave, titulo, autor, genero, isbn, año_publicacion,  fecha_adquisicion))
      print(f"=Clave: {clave}\n=Titulo: {titulo}\n=Autor: {autor}\n=Genero: {genero}\n=Año de Publicación: {año_publicacion}\n=ISBN: {isbn}\n=Fecha: {fecha_adquisicion}")
      print(f"-"*30)  
      if not ejemplares_por_año_publicacion:
        print(f"No se encontraron ejemplares del Año {consulta_reporte}")
  libro = openpyxl.Workbook()
  hoja = libro["Sheet"]
  hoja.title = f"Ejemplares de {consulta_reporte}"

  hoja["A1"].value = "CLAVE"
  hoja["B1"].value = "TÍTULO"
  hoja["C1"].value = "AUTOR"
  hoja["D1"].value = "GÉNERO"
  hoja["E1"].value = "ISBN"
  hoja["F1"].value = "AÑO PUBLICACION"
  hoja["G1"].value = "FECHA"

  for row_num, ejemplar in enumerate(ejemplares_por_año_publicacion, 2):
    for col_num, valor in enumerate(ejemplar, 1):
        hoja.cell(row=row_num, column=col_num, value=valor)

  nombre_archivo = f"ejemplares_por_año_publicacion_{consulta_reporte}.xlsx"
  libro.save(nombre_archivo)
  print(f"Archivo guardado como {nombre_archivo}")


#Tabla a usar
ejemplares()

#Menús
import sys
import datetime
from datetime import date, datetime
import re
import openpyxl
import csv
import sqlite3
from sqlite3 import Error

while True:
  print("="*30)
  print("-----MENU PRINCIPAL-----")
  print("="*30)
  opcion_principal=int(input("= Ingrese la opción que desee ejecutar:\n\
= Opción 1: Registrar ejemplar\n\
= Opción 2: Registrar Autor\n\
= Opción 3: Registrar Género\n\
= Opción 4: Agregar Autor/Género a ejemplar\n\
= Opción 5: Menu de Consultas y Reportes\n\
= Opción 6: Salir del Programa\n> "))

  #Registrar ejemplar preguntando sus datos
  if opcion_principal == 1:
    while True:
      while True:
        titulo=input("= Ingrese el título del nuevo ejemplar:\n> ").upper()
        if len(titulo) < 3:
          print(f"**El título: {titulo} proporcionado no puede tener menos de 3 carácteres**")
        else:
          print("---Título guardado---\n")
          break
          
      #Comprobar formato del Año de Publicación el Ejemplar
      while True:
        año_publicacion=input(f"= Ingrese el año de publicación del ejemplar:\n> ")
        if not(bool(re.match("^[0-9]{4}$",año_publicacion))):
          print(f"**El año: {año_publicacion}, no cumple con el formato de digítos, pruebe de nuevo**\n")
        elif año_publicacion > '2023':
          print(f"**El año {año_publicacion} aún no pasa, intenté de nuevo.**\n")
          continue
        else:
          print("---Año guardado---\n")
          break

      #Comprobar formato del ISBN del Ejemplar
      while True:
        isbn=input("= Ingrese el ISBN del ejemplar con el siguiente formato (000-00-0000-000-0):\n> ")
        if bool(re.match("^[0-9]{3}[-][0-9]{2}[-][0-9]{4}[-][0-9]{3}[-][0-9]{1}$",isbn)):
          print("---ISBN guardado---\n")
          break
        else:
          print(f"**El ISBN: {isbn}, no cumple con el formato de digítos, pruebe de nuevo**\n")
          continue

      #Comprobar formato de la Fecha de Adquisición del Ejemplar
      while True:
        fecha_adquisicion=input("= Ingrese la fecha de adquisición del ejemplar con el siguiente formato(dd/mm/aaaa):\n> ")
        fecha_actual = datetime.today()
        if not(bool(re.match("^[0-9]{2}[/][0-9]{2}[/][0-9]{4}$", fecha_adquisicion))):
          print(f"**La fecha de ingresó no cumple con el formato, intenté de nuevo**\n")
          continue
        try:
          fecha_datetime = datetime.strptime(fecha_adquisicion, '%d/%m/%Y')
          if fecha_datetime > fecha_actual:
            print(f"**La fecha {fecha_adquisicion} aún no ha llegado.**\n")
          else:
            print("---Fecha de adquisión guardado---\n")
            break
        except ValueError:
          print(f"**La fecha {fecha_adquisicion} no existe, intenté de nuevo.**\n")

      registrar_otro=input("= ¿Desea agregar otro ejemplar?: S/N\n> ").upper()
      if registrar_otro == 'S':
        continue
      elif registrar_otro == 'N':
      # Insertar el ejemplar en la tabla TABLA_EJEMPLARES
        try:
          with sqlite3.connect("EV3_EJEMPLARES.db") as conn:
            cursor = conn.cursor()
            valores = (titulo, autor_id, genero_id, año_publicacion, isbn, fecha_adquisicion)
            cursor.execute("INSERT INTO TABLA_EJEMPLARES \
            (titulo, AUTORES, GENERO, año_publicacion, isbn, fecha_adquisicion) VALUES (?, ?, ?, ?, ?, ?)", valores)
            ejemplar_id = cursor.lastrowid
            print(f"== El ejemplar {titulo} se ha registrado con éxito con la clave de ejemplar: {ejemplar_id} ==")
            print("---Ejemplar registrado exitosamente---\n")
        except Error as e:
          print(e)
        except:
          print(f"**Se produjo el siguiente error: {sys.exc_info()[0]}**")
        break
      else:
        print(f"**La opción: {registrar_otro} no cumple con el formato de la pregunta.**")

  #Registrar Autor
  if opcion_principal == 2:
    while True:
      # Registrar Autor
      while True:
        nombre_autor=input(f"= Ingrese el nombre del autor:\n> ").upper()
        if len(nombre_autor) < 3:
          print(f"**El nombre del autor: {nombre_autor} proporcionado no puede tener menos de 3 caracteres**")
        else:
          print("---Nombre de autor guardado---\n")
          break

      while True:
        apellido_autor=input(f"= Ingrese el apellido del autor {nombre_autor}:\n> ").upper()
        if len(apellido_autor) < 3:
          print(f"**El apellido del autor: {apellido_autor} proporcionado no puede tener menos de 3 caracteres**")
        else:
          print("---Apellido de autor guardado---\n")
          break

      # Insertar autor en la tabla AUTORES
      try:
        with sqlite3.connect("EV3_EJEMPLARES.db") as conn:
          cursor = conn.cursor()
          cursor.execute("INSERT INTO AUTORES (apellido, nombre) VALUES (?, ?)", (apellido_autor, nombre_autor))
          autor_id = cursor.lastrowid
          print(f"== El autor {nombre_autor} {apellido_autor} se ha registrado con éxito con la clave de autor: {autor_id} ==")
          print("---Autor registrado exitosamente---\n")
      except Error as e:
        print(e)
      except:
        print(f"**Se produjo el siguiente error: {sys.exc_info()[0]}**")
      
      registrar_otro=input("= ¿Desea agregar otro ejemplar?: S/N\n> ").upper()
      if registrar_otro == 'S':
        continue
      elif registrar_otro == 'N':
        break
      else:
        print(f"**La opción: {registrar_otro} no cumple con el formato de la pregunta.**")

  #Registrar Género
  if opcion_principal == 3:
    # Registrar Género
    while True:
      genero=input(f"= Ingrese el tipo de género que tiene el ejemplar:\n> ").upper()
      if len(genero) < 3:
        print(f"**El género: {genero} proporcionado no puede tener menos de 3 caracteres**")
      else:
        print("---Género guardado---\n")
        break

    # Insertar género en la tabla GENERO
    try:
      with sqlite3.connect("EV3_EJEMPLARES.db") as conn:
        cursor = conn.cursor()
        cursor.execute("INSERT INTO GENERO (genero) VALUES (?)", (genero,))
        genero_id = cursor.lastrowid
        print(f"== El género {genero} se ha registrado con éxito con la clave de género: {genero_id} ==")
        print("---Género registrado exitosamente---\n")
    except Error as e:
      print(e)
    except:
      print(f"**Se produjo el siguiente error: {sys.exc_info()[0]}**")

    registrar_otro=input("= ¿Desea agregar otro ejemplar?: S/N\n> ").upper()
    if registrar_otro == 'S':
      continue
    elif registrar_otro == 'N':
      break
    else:
      print(f"**La opción: {registrar_otro} no cumple con el formato de la pregunta.**")

  if opcion_principal == 4:
    while True:
      opcion_agregar=int(input("= Ingrese la opción para agregar el valor:\n\
    = Opción 1: Agregar autor a un ejemplar\n\
    = Opción 2: Agregar género a un ejemplar\n\
    = Opción 3: Volver\n> "))
      if opcion_agregar == 1:
        #Insertar Autor a un Ejemplar
        try:
            with sqlite3.connect("EV3_EJEMPLARES.db") as conn:
                cursor = conn.cursor()
                
                # Mostrar la lista de autores existentes
                cursor.execute("SELECT clave_autor, apellido, nombre FROM AUTORES")
                autores = cursor.fetchall()
                print("=== Autores existentes ===")
                for clave_autor, apellido, nombre in autores:
                    print(f"Clave: {clave_autor}, Apellido: {apellido}, Nombre: {nombre}")
                print("="*50)
                
                # Obtener la clave del autor que se desea agregar al ejemplar
                while True:
                    clave_autor = input("Ingrese la clave del autor que desea agregar al ejemplar:\n> ")
                    cursor.execute("SELECT clave_autor FROM AUTORES WHERE clave_autor = ?", (clave_autor,))
                    autor_existente = cursor.fetchall()
                    if autor_existente:
                        break
                    else:
                        print(f"**La clave del autor {clave_autor} no existe, intenta de nuevo**")
                
                # Obtener la clave del ejemplar al que se desea agregar el autor
                while True:
                    # Mostrar la lista de ejemplares existentes
                    cursor.execute("SELECT clave, titulo FROM TABLA_EJEMPLARES")
                    ejemplares_existentes = cursor.fetchall()
                    print("=== Ejemplares existentes ===")
                    for clave, titulo in ejemplares_existentes:
                        print(f"Clave: {clave}, Título: {titulo}")
                    print("="*50)
                    clave_ejemplar = input("Ingrese la clave del ejemplar al que desea agregar el autor:\n> ")
                    cursor.execute("SELECT clave FROM TABLA_EJEMPLARES WHERE clave = ?", (clave_ejemplar,))
                    ejemplar_existente = cursor.fetchall()
                    if ejemplar_existente:
                        break
                    else:
                        print(f"**La clave del ejemplar {clave_ejemplar} no existe, intenta de nuevo**")
                
                # Agregar el autor al ejemplar
                cursor.execute("UPDATE TABLA_EJEMPLARES SET AUTORES = ? WHERE clave = ?", (clave_autor, clave_ejemplar))
                conn.commit()
                print("--- Autor agregado al ejemplar exitosamente ---")
                
        except sqlite3.Error as e:
            print(e)
        except Exception:
            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
        finally:
            if conn:
                conn.close()
                print("Se ha cerrado la conexión")
        break

      if opcion_agregar == 2:
        #Insertar Género a un Ejemplar
        try:
            with sqlite3.connect("EV3_EJEMPLARES.db") as conn:
                cursor = conn.cursor()
                
                # Mostrar la lista de géneros existentes
                cursor.execute("SELECT clave_genero, genero FROM GENERO")
                genero = cursor.fetchall()
                print("=== Géneros existentes ===")
                for clave_genero, genero in genero:
                    print(f"Clave: {clave_genero}, Género: {genero}")
                print("="*50)
                
                # Obtener la clave del genero que se desea agregar al ejemplar
                while True:
                    clave_genero = input("Ingrese la clave del género que desea agregar al ejemplar:\n> ")
                    cursor.execute("SELECT clave_genero FROM GENERO WHERE clave_genero = ?", (clave_genero,))
                    genero_existente = cursor.fetchall()
                    if genero_existente:
                        break
                    else:
                        print(f"**La clave del género {clave_genero} no existe, intenta de nuevo**")
                
                # Obtener la clave del ejemplar al que se desea agregar el género
                while True:
                    # Mostrar la lista de ejemplares existentes
                    cursor.execute("SELECT clave, titulo FROM TABLA_EJEMPLARES")
                    ejemplares_existentes = cursor.fetchall()
                    print("=== Ejemplares existentes ===")
                    for clave, titulo in ejemplares_existentes:
                        print(f"Clave: {clave}, Título: {titulo}")
                    print("="*50)
                    clave_ejemplar = input("Ingrese la clave del ejemplar al que desea agregar el género:\n> ")
                    cursor.execute("SELECT clave FROM TABLA_EJEMPLARES WHERE clave = ?", (clave_ejemplar,))
                    ejemplar_existente = cursor.fetchall()
                    if ejemplar_existente:
                        break
                    else:
                        print(f"**La clave del ejemplar {clave_ejemplar} no existe, intenta de nuevo**")
                
                # Agregar el autor al ejemplar
                cursor.execute("UPDATE TABLA_EJEMPLARES SET GENERO = ? WHERE clave = ?", (clave_genero, clave_ejemplar))
                conn.commit()
                print("--- Género agregado al ejemplar exitosamente ---")
                
        except sqlite3.Error as e:
            print(e)
        except Exception:
            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
        finally:
            if conn:
                conn.close()
                print("Se ha cerrado la conexión")
        break
      else:
        print("La opción no existe, intenté de nuevo")
        continue

  #Menú consultas y reportes
  elif opcion_principal == 4:
    while True:  
      print("="*40)
      print("-----MENU CONSULTAS Y REPORTES-----")
      print("="*40)
      opcion_consultas_reportes=int(input("= Ingrese la opción que desee ejecutar:\n\
= Opción 1: Consultas\n\
= Opción 2: Reportes\n\
= Opción 3: Volver al Menú Principal\n> "))

      if opcion_consultas_reportes == 1:
        while True:
          print("="*30)
          print("-----MENU CONSULTAS-----")
          print("="*30)
          opcion_consultas=int(input("= Ingrese la opción que desee ejecutar:\n\
= Opción 1: Consulta por medio de Título\n\
= Opción 2: Consulta por medio de ISBN\n\
= Opción 3: Volver al Menú del Consultas y Reportes\n> "))
          if opcion_consultas == 1:
            #Consultar ejemplares por medio de Título
            try:
                with sqlite3.connect("EV3_EJEMPLARES.db") as conn:
                    cursor = conn.cursor()
                    cursor.execute("SELECT clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion FROM TABLA_EJEMPLARES")
                    registros = cursor.fetchall()

                    lista_titulo=[titulo for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros]
                    while True:
                      print(f"=== Títulos de ejemplares existentes ===")
                      print(f"{lista_titulo}")
                      consulta_reporte=input("= Ingrese el título que desee consultar para mostrar sus datos:\n> ").upper()
                      
                      if consulta_reporte in lista_titulo:
                        break
                      else:
                        print(f"**El Título {consulta_reporte} no existé, intenté de nuevo**")
                        continue
                    
                    cursor.execute("SELECT clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion FROM TABLA_EJEMPLARES \
                    WHERE upper(titulo)=?", (consulta_reporte,))
                    registros_titulo = cursor.fetchall()
                        
                    print("="*50)
                    print("----CONSULTA DE EJEMPLARES POR MEDIO DE TITULO----")
                    print("="*50)      
                    for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros_titulo:
                      print(f"= Clave: {clave}\n= Titulo: {titulo}\n= Autor: {autor}\
                      \n= Genero: {genero}\n= ISBN: {isbn}\n= Año de Publicación: {año_publicacion}\
                      \n= Fecha de adquisición = {fecha_adquisicion}")
                      print("-"*50)

            except sqlite3.Error as e:
                print (e)
            except Exception:
                print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
            finally:
                if (conn):
                    conn.close()
                    print("Se ha cerrado la conexión")
          elif opcion_consultas == 2:
            #Consultar ejemplares por medio del ISBN
            try:
                with sqlite3.connect("EV3_EJEMPLARES.db", detect_types = sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES) as conn:
                    cursor = conn.cursor()
                    cursor.execute("SELECT clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion FROM TABLA_EJEMPLARES")
                    registros = cursor.fetchall()

                    lista_isbn=[isbn for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros]
                    while True:
                      print(f"=== ISBN de ejemplares existentes ===")
                      print(f"{lista_isbn}")
                      consulta_reporte=input("= Ingrese el título que desee consultar para mostrar sus datos:\n> ").upper()
                      
                      if consulta_reporte in lista_isbn:
                        break
                      else:
                        print(f"**El ISBN {consulta_reporte} no existé, intenté de nuevo**")
                        continue
                    
                    cursor.execute("SELECT clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion FROM TABLA_EJEMPLARES \
                    WHERE upper(isbn)=?", (consulta_reporte,))
                    registros_isbn = cursor.fetchall()
                        
                    print("="*50)
                    print("----CONSULTA DE EJEMPLARES POR MEDIO DE ISBN----")
                    print("="*50)      
                    for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros_isbn:
                      print(f"= Clave: {clave}\n= Titulo: {titulo}\n= Autor: {autor}\
                      \n= Genero: {genero}\n= Año de Publicación: {año_publicacion}\n= ISBN: {isbn}\
                      \n= Fecha de adquisición = {fecha_adquisicion.date().strftime('%d/%m/%Y')}")
                      print("-"*50)

            except sqlite3.Error as e:
                print (e)
            except Exception:
                print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
            finally:
                if (conn):
                    conn.close()
                    print("Se ha cerrado la conexión") 
          elif opcion_consultas == 3:
            break
          else:
            print("La opción que ingresó no cumple con el formato, intenté de nuevo")       
      elif opcion_consultas_reportes == 2:
        while True:
          print("="*30)
          print("-----MENU REPORTES-----")
          print("="*30)
          opcion_reportes=int(input("= Ingrese la opción que desee ejecutar:\n\
= Opción 1: Catálago Completo\n\
= Opción 2: Reporte por Autor\n\
= Opción 3: Reporte por Género\n\
= Opción 4: Reporte por Año de Publicación\n\
= Opción 5: Volver al Menú de consultas y reportes\n> "))
          if opcion_reportes == 1:
            #Consultar ejemplares por medio de Catálogo Completo
            try:
                with sqlite3.connect("EV3_EJEMPLARES.db", detect_types = sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES) as conn:
                  cursor = conn.cursor()
                  cursor.execute("SELECT clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion FROM TABLA_EJEMPLARES")
                  registros = cursor.fetchall()
                  print("="*40)
                  print("----CATÁLOGO COMPLETO DE EJEMPLARES----")
                  print("="*40)
                  for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros:
                    print(f"= Clave: {clave}\n= Titulo: {titulo}\n= Autor: {autor}\
                    \n= Genero: {genero}\n= Año de Publicación: {año_publicacion}\n= ISBN: {isbn}\
                    \n= Fecha de adquisición = {fecha_adquisicion.date().strftime('%d/%m/%Y')}")
                    print("-"*40)
            except sqlite3.Error as e:
              print (e)
            except Exception:
              print(f"**Se produjo el siguiente error: {sys.exc_info()[0]}**")
            finally:
              if (conn):
                conn.close()
                print("---Se ha cerrado la conexión---")

            while True:
              print("="*40)
              print("-----MENU EXTRACCION-----")
              print("="*40)
              opcion_extraer = int(input("= Ingrese el tipo de extracción del reporte/consulta que desee ejecutar:\n\
= Opción 1: Extraer a CSV\n\
= Opción 2: Extraer a Excel\n\
= Opción 3: Volver al Menú de Reportes\n> "))

              if opcion_extraer == 1:
                ejemplares_csv()

              elif opcion_extraer == 3:
                break

              if opcion_extraer == 2:
                ejemplares_excel()

              if opcion_extraer == 3:
                break

          elif opcion_reportes == 2:
            #Reporte de ejemplares por medio de Autor
            try:
                with sqlite3.connect("EV3_EJEMPLARES.db", detect_types = sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES) as conn:
                    cursor = conn.cursor()
                    cursor.execute("SELECT clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion FROM TABLA_EJEMPLARES")
                    registros = cursor.fetchall()

                    lista_autores=[autor for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros]
                    autores_unicos=set(lista_autores)
                    while True:
                      print(f"=== Autores de ejemplares existentes ===")
                      print(f"{autores_unicos}")
                      consulta_reporte=input("= Ingrese el autor que desee consultar para mostrar sus datos:\n> ").upper()
                      
                      if consulta_reporte in autores_unicos:
                        break
                      else:
                        print(f"**El Título {consulta_reporte} no existé, intenté de nuevo**")
                        continue
                    
                    cursor.execute("SELECT clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion FROM TABLA_EJEMPLARES \
                    WHERE upper(autor)=?", (consulta_reporte,))
                    registros_titulo = cursor.fetchall()
                        
                    print("="*50)
                    print("----REPORTE DE EJEMPLARES POR MEDIO DE AUTOR----")
                    print("="*50)      
                    for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros_titulo:
                      print(f"= Clave: {clave}\n= Titulo: {titulo}\n= Autor: {autor}\
                      \n= Genero: {genero}\n= Año de Publicación: {año_publicacion}\n= ISBN: {isbn}\
                      \n= Fecha de adquisición = {fecha_adquisicion.date().strftime('%d/%m/%Y')}")
                      print("-"*50)

            except sqlite3.Error as e:
                print (e)
            except Exception:
                print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
            finally:
                if (conn):
                    conn.close()
                    print("Se ha cerrado la conexión")
            while True:
              print("="*40)
              print("-----MENU EXTRACCION-----")
              print("="*40)
              opcion_extraer = int(input("= Ingrese el tipo de extracción del reporte/consulta que desee ejecutar:\n\
= Opción 1: Extraer a CSV\n\
= Opción 2: Extraer a Excel\n\
= Opción 3: Volver al Menú de Reportes\n> "))
              if opcion_extraer == 1:
                csv_autor()
              if opcion_extraer == 2:
                excel_autor()
              if opcion_extraer == 3:
                break
                  
          elif opcion_reportes == 3:
            #Reporte de ejemplares por medio de Género
            try:
                with sqlite3.connect("EV3_EJEMPLARES.db", detect_types = sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES) as conn:
                    cursor = conn.cursor()
                    cursor.execute("SELECT clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion FROM TABLA_EJEMPLARES")
                    registros = cursor.fetchall()

                    lista_generos=[genero for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros]
                    generos_unicos=set(lista_generos)
                    while True:
                      print(f"=== Géneros de ejemplares existentes ===")
                      print(f"{generos_unicos}")
                      consulta_reporte=input("= Ingrese el género que desee consultar para mostrar sus datos:\n> ").upper()
                      
                      if consulta_reporte in generos_unicos:
                        break
                      else:
                        print(f"**El Género {consulta_reporte} no existé, intenté de nuevo**")
                        continue
                    
                    cursor.execute("SELECT clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion FROM TABLA_EJEMPLARES \
                    WHERE upper(genero)=?", (consulta_reporte,))
                    registros_generos = cursor.fetchall()
                        
                    print("="*50)
                    print("----REPORTE DE EJEMPLARES POR MEDIO DE GENEROS----")
                    print("="*50)      
                    for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros_generos:
                      print(f"= Clave: {clave}\n= Titulo: {titulo}\n= Autor: {autor}\
                      \n= Genero: {genero}\n= Año de Publicación: {año_publicacion}\n= ISBN: {isbn}\
                      \n= Fecha de adquisición = {fecha_adquisicion.date().strftime('%d/%m/%Y')}")
                      print("-"*50)

            except sqlite3.Error as e:
                print (e)
            except Exception:
                print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
            finally:
                if (conn):
                    conn.close()
                    print("Se ha cerrado la conexión")

            while True:
              print("="*40)
              print("-----MENU EXTRACCION-----")
              print("="*40)
              opcion_extraer = int(input("= Ingrese el tipo de extracción del reporte/consulta que desee ejecutar:\n\
= Opción 1: Extraer a CSV\n\
= Opción 2: Extraer a Excel\n\
= Opción 3: Volver al Menú de Reportes\n> "))
              if opcion_extraer == 2:
                excel_genero()
              if opcion_extraer == 3:
                break

          elif opcion_reportes == 4:
            #Reporte de ejemplares por medio de Año de Publicación
            try:
                with sqlite3.connect("EV3_EJEMPLARES.db", detect_types = sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES) as conn:
                    cursor = conn.cursor()
                    cursor.execute("SELECT clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion FROM TABLA_EJEMPLARES")
                    registros = cursor.fetchall()

                    lista_año_publicacion=[año_publicacion for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros]
                    año_publicacion_unicos=set(lista_año_publicacion)
                    while True:
                      print(f"=== Géneros de ejemplares existentes ===")
                      print(f"{año_publicacion_unicos}")
                      consulta_reporte=input("= Ingrese el año de publicación que desee consultar para mostrar sus datos:\n> ").upper()
                      
                      if consulta_reporte in año_publicacion_unicos:
                        break
                      else:
                        print(f"**El Año de publicación {consulta_reporte} no existé, intenté de nuevo**")
                        continue
                    
                    cursor.execute("SELECT clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion FROM TABLA_EJEMPLARES \
                    WHERE upper(año_publicacion)=?", (consulta_reporte,))
                    registros_año_publicacion = cursor.fetchall()
                        
                    print("="*50)
                    print("----REPORTE DE EJEMPLARES POR MEDIO DE AÑO DE PUBLICACIÓN----")
                    print("="*50)      
                    for clave, titulo, autor, genero, isbn, año_publicacion, fecha_adquisicion in registros_año_publicacion:
                      print(f"= Clave: {clave}\n= Titulo: {titulo}\n= Autor: {autor}\
                      \n= Genero: {genero}\n= Año de Publicación: {año_publicacion}\n= ISBN: {isbn}\
                      \n= Fecha de adquisición = {fecha_adquisicion.date().strftime('%d/%m/%Y')}")
                      print("-"*50)

            except sqlite3.Error as e:
                print (e)
            except Exception:
                print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
            finally:
                if (conn):
                    conn.close()
                    print("Se ha cerrado la conexión")
  
            while True:
              print("="*40)
              print("-----MENU EXTRACCION-----")
              print("="*40)
              opcion_extraer = int(input("= Ingrese el tipo de extracción del reporte/consulta que desee ejecutar:\n\
= Opción 1: Extraer a CSV\n\
= Opción 2: Extraer a Excel\n\
= Opción 3: Volver al Menú de Reportes\n> "))
              if opcion_extraer == 2:
                excel_año_publicacion()

              if opcion_extraer == 3:
                break

          elif opcion_reportes == 5:
            break
          else:
            print("La opción que ingresó no cumple con el formato, intenté de nuevo")

      elif opcion_consultas_reportes == 3:
        break
      else:
        print("La opción que ingresó no cumple con el formato, intenté de nuevo")

  elif opcion_principal == 6:
    break